<a href="https://colab.research.google.com/github/imaaditya-stack/SpamFilterForQuoraQuestions-DeepLearning/blob/master/SpamFilterForQuora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/spamfilterdata.csv")

In [3]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [4]:
data.shape

(1306122, 3)

In [5]:
data.target.value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [0]:
data.drop(columns="qid", inplace=True)

In [7]:
data.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [0]:
data["question_text_length"] = data.question_text.apply(lambda x: len(x.split(" ")))

In [9]:
data.head()

,question_text,target,question_text_length
0,How did Quebec nationalists see their province...,0,13
1,"Do you have an adopted dog, how would you enco...",0,16
2,Why does velocity affect time? Does velocity a...,0,10
3,How did Otto von Guericke used the Magdeburg h...,0,9
4,Can I convert montra helicon D to a mountain b...,0,15


In [10]:
max(data.question_text_length), min(data.question_text_length)

(122, 1)

In [0]:
print(list(data["question_text"][data["question_text_length"]==122].values))

['In "Star Trek 2013" why did they :\n\n*Spoilers*\n*Spoilers*\n*Spoilers*\n*Spoilers*\n\n1)Make warping look quite a bit like an hyperspace jump\n2)what in the world were those bright particles as soon as they jumped.\n3)Why in the world did they make it possible for two entities to react in warp space in separate jumps.\n4)Why did Spock get emotions for this movie.\n5)What was the point of hiding the "Enterprise" underwater.\n6)When they were intercepted by the dark ship, how come they reached Earth when they were far away from her.(I don\'t seem to remember the scene where they warp to earth).\n7)How did the ship enter earth\'s atmosphere when it wasnt even in orbit.\n8)When Scotty opened the door of the black ship , how come pike and khan didn\'t slow down?']


In [14]:
sampleText = list(data["question_text"][data["question_text_length"]==122].values)[0]
sampleText

'In "Star Trek 2013" why did they :\n\n*Spoilers*\n*Spoilers*\n*Spoilers*\n*Spoilers*\n\n1)Make warping look quite a bit like an hyperspace jump\n2)what in the world were those bright particles as soon as they jumped.\n3)Why in the world did they make it possible for two entities to react in warp space in separate jumps.\n4)Why did Spock get emotions for this movie.\n5)What was the point of hiding the "Enterprise" underwater.\n6)When they were intercepted by the dark ship, how come they reached Earth when they were far away from her.(I don\'t seem to remember the scene where they warp to earth).\n7)How did the ship enter earth\'s atmosphere when it wasnt even in orbit.\n8)When Scotty opened the door of the black ship , how come pike and khan didn\'t slow down?'

In [12]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
word_tokenize(sampleText.lower())

['in',
 '``',
 'star',
 'trek',
 '2013',
 "''",
 'why',
 'did',
 'they',
 ':',
 '*spoilers*',
 '*spoilers*',
 '*spoilers*',
 '*spoilers*',
 '1',
 ')',
 'make',
 'warping',
 'look',
 'quite',
 'a',
 'bit',
 'like',
 'an',
 'hyperspace',
 'jump',
 '2',
 ')',
 'what',
 'in',
 'the',
 'world',
 'were',
 'those',
 'bright',
 'particles',
 'as',
 'soon',
 'as',
 'they',
 'jumped',
 '.',
 '3',
 ')',
 'why',
 'in',
 'the',
 'world',
 'did',
 'they',
 'make',
 'it',
 'possible',
 'for',
 'two',
 'entities',
 'to',
 'react',
 'in',
 'warp',
 'space',
 'in',
 'separate',
 'jumps',
 '.',
 '4',
 ')',
 'why',
 'did',
 'spock',
 'get',
 'emotions',
 'for',
 'this',
 'movie',
 '.',
 '5',
 ')',
 'what',
 'was',
 'the',
 'point',
 'of',
 'hiding',
 'the',
 '``',
 'enterprise',
 "''",
 'underwater',
 '.',
 '6',
 ')',
 'when',
 'they',
 'were',
 'intercepted',
 'by',
 'the',
 'dark',
 'ship',
 ',',
 'how',
 'come',
 'they',
 'reached',
 'earth',
 'when',
 'they',
 'were',
 'far',
 'away',
 'from',
 'her',

Numbers removing

In [16]:
import re
result = re.sub(r'\d+','', sampleText)
result

'In "Star Trek " why did they :\n\n*Spoilers*\n*Spoilers*\n*Spoilers*\n*Spoilers*\n\n)Make warping look quite a bit like an hyperspace jump\n)what in the world were those bright particles as soon as they jumped.\n)Why in the world did they make it possible for two entities to react in warp space in separate jumps.\n)Why did Spock get emotions for this movie.\n)What was the point of hiding the "Enterprise" underwater.\n)When they were intercepted by the dark ship, how come they reached Earth when they were far away from her.(I don\'t seem to remember the scene where they warp to earth).\n)How did the ship enter earth\'s atmosphere when it wasnt even in orbit.\n)When Scotty opened the door of the black ship , how come pike and khan didn\'t slow down?'

In [0]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
tokens = word_tokenize(sampleText)
result = [i for i in tokens if not i in stop_words]
print(result)

NameError: ignored

In [0]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()
input_str=word_tokenize(sampleText)
for word in input_str:
    print(lemmatizer.lemmatize(word))

In
``
Star
Trek
2013
''
why
did
they
:
*Spoilers*
*Spoilers*
*Spoilers*
*Spoilers*
1
)
Make
warping
look
quite
a
bit
like
an
hyperspace
jump
2
)
what
in
the
world
were
those
bright
particle
a
soon
a
they
jumped
.
3
)
Why
in
the
world
did
they
make
it
possible
for
two
entity
to
react
in
warp
space
in
separate
jump
.
4
)
Why
did
Spock
get
emotion
for
this
movie
.
5
)
What
wa
the
point
of
hiding
the
``
Enterprise
''
underwater
.
6
)
When
they
were
intercepted
by
the
dark
ship
,
how
come
they
reached
Earth
when
they
were
far
away
from
her
.
(
I
do
n't
seem
to
remember
the
scene
where
they
warp
to
earth
)
.
7
)
How
did
the
ship
enter
earth
's
atmosphere
when
it
wasnt
even
in
orbit
.
8
)
When
Scotty
opened
the
door
of
the
black
ship
,
how
come
pike
and
khan
did
n't
slow
down
?


In [0]:
sampleText

'In "Star Trek 2013" why did they :\n\n*Spoilers*\n*Spoilers*\n*Spoilers*\n*Spoilers*\n\n1)Make warping look quite a bit like an hyperspace jump\n2)what in the world were those bright particles as soon as they jumped.\n3)Why in the world did they make it possible for two entities to react in warp space in separate jumps.\n4)Why did Spock get emotions for this movie.\n5)What was the point of hiding the "Enterprise" underwater.\n6)When they were intercepted by the dark ship, how come they reached Earth when they were far away from her.(I don\'t seem to remember the scene where they warp to earth).\n7)How did the ship enter earth\'s atmosphere when it wasnt even in orbit.\n8)When Scotty opened the door of the black ship , how come pike and khan didn\'t slow down?'

In [0]:
sentence = re.sub(r'[)\n*"?.,]',r'',sampleText)
sentence

"In Star Trek 2013 why did they :SpoilersSpoilersSpoilersSpoilers1Make warping look quite a bit like an hyperspace jump2what in the world were those bright particles as soon as they jumped3Why in the world did they make it possible for two entities to react in warp space in separate jumps4Why did Spock get emotions for this movie5What was the point of hiding the Enterprise underwater6When they were intercepted by the dark ship how come they reached Earth when they were far away from her(I don't seem to remember the scene where they warp to earth7How did the ship enter earth's atmosphere when it wasnt even in orbit8When Scotty opened the door of the black ship  how come pike and khan didn't slow down"

In [0]:
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in final_X:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # Stemming and removing stopwords
    temp.append(words)
    
final_X = temp   